In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.
/Users/mounirhader/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [35]:
# create RNN data
import os
import datetime

datafilestrain1 = os.listdir('data_complete')
datafiles = [a for a in datafilestrain1 if a.startswith('AS')]
datafiles.remove('AS14.32.csv')
print len(datafiles)

################

window=1

for user in datafiles:
    for fold in [1,2,3,4]:
        train_dataset  = pd.read_csv('data_complete/fold_{}/train/{}'.format(fold, user))
        test_dataset = pd.read_csv('data_complete/fold_{}/test/'.format(fold) + user)

        test_indices = list(test_dataset['Unnamed: 0'])
        print test_indices
        train_indices = list(train_dataset['Unnamed: 0'])
        print train_indices

        df = pd.concat([train_dataset, test_dataset])
        df = df.sort_values('Unnamed: 0')
        normalized_test_df = pd.read_csv('data_normalized/fold_{}/test/{}'.format(fold, user))
        normalized_train_df = pd.read_csv('data_normalized/fold_{}/train/{}'.format(fold, user))
        #print vars1
        #vars1.remove('Unnamed: 0')
        vars2 = ['predict_day', 'user', 'window', 'true_mood', 'indexje', 'weekday', 'weekend_day']
        cols=vars2+vars1

        dataset_df = pd.DataFrame(columns=cols)
        days = list(df['Unnamed: 0.1'])
        for i in range(len(df)):
            if i + window <= len(df) - 1:
                windowstart = days[i]
                windowend = days[i+window-1]
                predictday = days[i+window]

                day_ints = [int(x) for x in predictday.split('-')]
                datum = datetime.date(day_ints[0], day_ints[1], day_ints[2])
                if datum.weekday() in [4,5,6]:
                    weekenddag = 1
                    weekdag = 0
                elif datum.weekday() in [0,1,2,3]:
                    weekenddag = 0
                    weekdag = 1
                
                avg_mood = df[i:i+window]['mood']
                avg_arousal = df[i:i+window]['arousal']
                avg_valence = df[i:i+window]['valence']
                avg_activity = df[i:i+window]['activity']

                normalized_row = normalized_test_df.loc[normalized_test_df['Unnamed: 0'] == days[i]]
                if normalized_row.empty:
                    normalized_row = normalized_train_df.loc[normalized_train_df['Unnamed: 0'] == days[i]]

                vardict = {}
                for var in vars1:
                    vardict['predict_day'] = predictday
                    vardict['mood'] = float(avg_mood)
                    vardict['arousal'] = float(avg_arousal)
                    vardict['valence'] = float(avg_valence)
                    vardict['activity'] = float(avg_activity)
                    vardict['true_mood'] = list(df['mood'])[i+window]
                    vardict['indexje'] = list(df['Unnamed: 0'])[i]
                    vardict['user'] = user[:-4]
                    vardict['window'] = '{}-{}'.format(windowstart, windowend)
                    vardict['weekday'] = weekdag
                    vardict['weekend'] = weekenddag
                    if var not in ['mood', 'arousal', 'valence', 'activity', 'Unnamed: 0']:
                        vardict[var] = float(normalized_row[var])

                dataset_df = dataset_df.append(vardict, ignore_index=True)
        
        print dataset_df
        dataset_df.indexje = dataset_df.indexje.astype(int)
        dataset_df.index = dataset_df['indexje']

        test_data = dataset_df.loc[[a-window for a in test_indices]]
        train_data = dataset_df.loc[[a-window for a in train_indices]]

        train_data = train_data[pd.notnull(train_data['predict_day'])]
        test_data = test_data[pd.notnull(test_data['predict_day'])]
        train_data = train_data.drop('indexje', axis=1)
        test_data = test_data.drop('indexje', axis=1)

        train_data.to_csv('data_rnn_weekdays/fold_{}/train/{}'.format(fold, user), index=False)
        test_data.to_csv('data_rnn_weekdays/fold_{}/test/{}'.format(fold, user), index=False)
        
        print user, fold



26
[9, 10, 11, 12, 13, 14, 15, 16, 17]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
AS14.01.csv 1
[18, 19, 20, 21, 22, 23, 24, 25, 26]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
AS14.01.csv 2
[27, 28, 29, 30, 31, 32, 33, 34, 35]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 36, 37, 38, 39, 40, 41, 42, 43, 44]
AS14.01.csv 3
[36, 37, 38, 39, 40, 41, 42, 43, 44]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
AS14.01.csv 4
[8, 9, 10, 11, 12, 13]
[2, 3, 4, 5, 6, 7, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


KeyError: 'finance'

In [33]:
train_data

,predict_day,user,window,true_mood,weekday,weekend_day,mood,arousal,valence,activity,screen,communication,finance,office,social,travel,gsm,amusement,system,weekend
indexje,,,,,,,,,,,,,,,,,,,,
0,2014-03-22,AS14.01,2014-03-21-2014-03-21,6.40,0,NaN,6.20,0.200000,0.200000,0.134050,0.208089,0.072695,0.000573,0.001993,0.052182,0.010595,0.545455,0.011660,0.043264,1.0
1,2014-03-23,AS14.01,2014-03-22-2014-03-22,6.80,0,NaN,6.40,0.600000,0.500000,0.236880,0.071090,0.057441,0.000244,0.000000,0.005088,0.000432,0.363636,0.001080,0.009827,1.0
2,2014-03-24,AS14.01,2014-03-23-2014-03-23,6.00,1,NaN,6.80,0.200000,0.800000,0.142741,0.078401,0.060617,0.000502,0.000000,0.010426,0.000000,0.000000,0.001092,0.015235,0.0
3,2014-03-25,AS14.01,2014-03-24-2014-03-24,6.75,1,NaN,6.00,0.800000,0.000000,0.078961,0.174159,0.107299,0.000395,0.000035,0.037310,0.004859,0.909091,0.011308,0.012103,0.0
4,2014-03-26,AS14.01,2014-03-25-2014-03-25,6.60,1,NaN,6.75,0.500000,0.500000,0.098374,0.248557,0.118944,0.000498,0.000000,0.022216,0.000000,0.090909,0.000789,0.014533,0.0
5,2014-03-27,AS14.01,2014-03-26-2014-03-26,7.00,1,NaN,6.60,-0.200000,0.600000,0.101308,0.190090,0.104036,0.000606,0.000000,0.053149,0.000000,0.000000,0.010538,0.013899,0.0
6,2014-03-28,AS14.01,2014-03-27-2014-03-27,6.40,0,NaN,7.00,0.200000,0.800000,0.159511,0.201877,0.080284,0.000489,0.002112,0.010826,0.000548,0.272727,0.007400,0.016300,1.0
7,2014-03-29,AS14.01,2014-03-28-2014-03-28,8.00,0,NaN,6.40,-0.600000,0.600000,0.095698,0.056985,0.010033,0.001032,0.000000,0.005935,0.013114,0.454545,0.014334,0.119957,1.0
17,2014-04-08,AS14.01,2014-04-07-2014-04-07,6.60,1,NaN,6.50,-0.666667,0.250000,0.050671,0.113963,0.085127,0.000534,0.000198,0.017077,0.000697,0.090909,0.005175,0.013116,0.0
